In [4]:
from keras.optimizers import Adam
from keras import backend as K

from keras.models import Sequential
from keras.layers import  Flatten, Dense, Activation,Convolution2D,MaxPooling2D
import numpy as np

In [5]:
%%bash 
# mkdir resize

In [6]:
import os
from scipy import misc

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# def rgb2gray(rgb):
#     return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

# for i in os.listdir('spectogram/'):
#     img = mpimg.imread('spectogram/' + i)
#     gray = rgb2gray(img)
#     misc.imsave('grayscales/'+i, gray)

In [7]:
from PIL import Image
import os
import PIL
# for i in os.listdir('spectogram/'):
#     img = Image.open('spectogram/' + i).resize((128, 128), PIL.Image.ANTIALIAS)
#     img.save('resize/' + i)

In [8]:

# t=[]
# for i in os.listdir('resize/'):
#     t.append(i)

    

In [39]:
name_t=[]
for i in os.listdir('spectogram/'):
    name_t.append(i.rstrip('.png'))

In [10]:
import scipy as sc
from scipy.ndimage  import imread

In [11]:
from scipy.misc import imresize

In [14]:
t=[]
for i in os.listdir('spectogram/'):
    img =imread('spectogram/'+i)
    t.append(imresize(img,size=(128,128),interp='bilinear')[...,:3])


In [15]:
a=np.array(t)


In [16]:
a.shape

(30, 128, 128, 3)

In [17]:
a

array([[[[ 71, 154,  92],
         [ 89, 178,  84],
         [ 91, 179,  82],
         ..., 
         [ 48, 162, 107],
         [ 34, 143, 118],
         [ 55, 118, 128]],

        [[ 78, 171, 103],
         [ 99, 200,  93],
         [103, 201,  91],
         ..., 
         [ 55, 182, 119],
         [ 37, 165, 131],
         [ 56, 147, 143]],

        [[ 66, 163, 111],
         [ 91, 197,  98],
         [ 87, 195, 100],
         ..., 
         [ 60, 185, 116],
         [ 36, 163, 132],
         [ 56, 149, 142]],

        ..., 
        [[117, 181,  79],
         [144, 213,  66],
         [133, 209,  72],
         ..., 
         [ 79, 194, 105],
         [ 47, 176, 124],
         [ 55, 159, 140]],

        [[128, 188,  72],
         [155, 216,  60],
         [157, 215,  58],
         ..., 
         [ 90, 198,  99],
         [ 53, 179, 121],
         [ 56, 159, 140]],

        [[107, 169,  74],
         [129, 191,  66],
         [139, 193,  59],
         ..., 
         [ 86, 178,  92],
  

In [18]:
from keras.layers import Input, Dense
from keras.models import Model

In [19]:
# this is the size of our encoded representations


encoding_dim =2048 # floats -> compression of factor 24, assuming the input is 49152 floats (49152)/24  ;128*128*3

# this is our input placeholder
input_img = Input(shape=(49152,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(49152, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [20]:
encoder = Model(input_img, encoded)

In [21]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [22]:
#autoencoder.sa('model1.h5')

In [23]:
X=a.astype('float32')/255.
X =X.reshape((len(X), np.prod(X.shape[1:])))

In [24]:
print (X.shape)

(30, 49152)


In [26]:
X

array([[ 0.27843139,  0.60392159,  0.36078432, ...,  0.21176471,
         0.56862748,  0.49803922],
       [ 0.18039216,  0.59215689,  0.39215687, ...,  0.50588238,
         0.7019608 ,  0.64313728],
       [ 0.3019608 ,  0.6156863 ,  0.34509805, ...,  0.76862746,
         0.79215688,  0.80000001],
       ..., 
       [ 0.27843139,  0.60392159,  0.36078432, ...,  0.21176471,
         0.56862748,  0.49803922],
       [ 0.41960785,  0.6901961 ,  0.25098041, ...,  0.13333334,
         0.48627451,  0.46666667],
       [ 0.8392157 ,  0.8392157 ,  0.8392157 , ...,  0.4509804 ,
         0.69803923,  0.25490198]], dtype=float32)

In [25]:
X

array([[ 0.27843139,  0.60392159,  0.36078432, ...,  0.21176471,
         0.56862748,  0.49803922],
       [ 0.18039216,  0.59215689,  0.39215687, ...,  0.50588238,
         0.7019608 ,  0.64313728],
       [ 0.3019608 ,  0.6156863 ,  0.34509805, ...,  0.76862746,
         0.79215688,  0.80000001],
       ..., 
       [ 0.27843139,  0.60392159,  0.36078432, ...,  0.21176471,
         0.56862748,  0.49803922],
       [ 0.41960785,  0.6901961 ,  0.25098041, ...,  0.13333334,
         0.48627451,  0.46666667],
       [ 0.8392157 ,  0.8392157 ,  0.8392157 , ...,  0.4509804 ,
         0.69803923,  0.25490198]], dtype=float32)

In [27]:
autoencoder.fit(X,X,
                epochs=20,
                batch_size=9,
                shuffle=True,
                )

Epoch 1/20
30/30 [==============================] - 20s - loss: 0.6945    
Epoch 2/20
30/30 [==============================] - 15s - loss: 0.6920    
Epoch 3/20
30/30 [==============================] - 15s - loss: 0.6907    
Epoch 4/20
30/30 [==============================] - 15s - loss: 0.6893    
Epoch 5/20
30/30 [==============================] - 15s - loss: 0.6874    
Epoch 6/20
30/30 [==============================] - 15s - loss: 0.6845    
Epoch 7/20
30/30 [==============================] - 15s - loss: 0.6804    
Epoch 8/20
30/30 [==============================] - 15s - loss: 0.6748    
Epoch 9/20
30/30 [==============================] - 15s - loss: 0.6677    
Epoch 10/20
30/30 [==============================] - 15s - loss: 0.6591    
Epoch 11/20
30/30 [==============================] - 15s - loss: 0.6498    
Epoch 12/20
30/30 [==============================] - 15s - loss: 0.6406    
Epoch 13/20
30/30 [==============================] - 15s - loss: 0.6326    
Epoch 14/20
30/30 [==

In [28]:
# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(X)
decoded_imgs = decoder.predict(encoded_imgs)

In [30]:
decoded_imgs

array([[ 0.37712276,  0.65477037,  0.37507817, ...,  0.40305248,
         0.69934446,  0.51031154],
       [ 0.3721261 ,  0.65871048,  0.3712796 , ...,  0.39799187,
         0.7082305 ,  0.50807333],
       [ 0.3808071 ,  0.65764654,  0.37713951, ...,  0.40493515,
         0.68339366,  0.5092923 ],
       ..., 
       [ 0.37712276,  0.65477037,  0.37507817, ...,  0.40305248,
         0.69934446,  0.51031154],
       [ 0.37313569,  0.65959954,  0.36402076, ...,  0.38908628,
         0.71868795,  0.517075  ],
       [ 0.39120716,  0.64875758,  0.36953223, ...,  0.40554988,
         0.69413459,  0.50237548]], dtype=float32)

In [32]:
import pandas as pd

    
    

In [32]:
import pandas as pd

In [33]:
c=pd.DataFrame(decoded_imgs,index=None)

    

In [34]:
c.shape

(30, 49152)

In [35]:
c.head()

,0,1,2,3,4,5,6,7,8,9,...,49142,49143,49144,49145,49146,49147,49148,49149,49150,49151
0,0.377123,0.654770,0.375078,0.412867,0.673108,0.342779,0.399932,0.718035,0.333365,0.382433,...,0.333496,0.460302,0.702895,0.370158,0.390452,0.689041,0.463589,0.403052,0.699344,0.510312
1,0.372126,0.658710,0.371280,0.411623,0.673374,0.346420,0.391871,0.725550,0.321411,0.378109,...,0.328509,0.456661,0.707566,0.366236,0.389235,0.699207,0.465402,0.397992,0.708230,0.508073
2,0.380807,0.657647,0.377140,0.410791,0.666152,0.357431,0.405265,0.713739,0.333283,0.381318,...,0.339167,0.459422,0.697785,0.370930,0.395673,0.690484,0.460174,0.404935,0.683394,0.509292
3,0.380448,0.655098,0.374034,0.411536,0.673598,0.344389,0.396461,0.716538,0.333569,0.382530,...,0.332566,0.460200,0.702658,0.367398,0.387203,0.692189,0.464573,0.403104,0.699180,0.512570
4,0.379877,0.658427,0.370546,0.414500,0.668260,0.353002,0.405181,0.717680,0.331619,0.379366,...,0.343811,0.458288,0.700104,0.365139,0.393691,0.690549,0.458788,0.404434,0.693158,0.513625


In [40]:
c['Name']=name_t

In [41]:
# import sklearn
# from sklearn.metrics import *
# g=pd.DataFrame(sklearn.metrics.pairwise.pairwise_distances(c.iloc[2:,:],c.iloc[1:,1:], metric='euclidean') ,\
#     index=list(c["Name"].values),columns=list(c["Name"].values))

In [42]:
c["Name"].values

array(['07- Ottawan - Hands u',
       '12 Just An Illusion The Best Disco... Ever Disc 3',
       "07Unforgettable - Nat 'King' Cole", '17Paul Anka - Pa',
       '06Elvis Presley - Wooden Heart (Audio)',
       '11-El Perdón (Forgiveness) - Nicky Jam & Enrique Iglesias _ Video Lyrics',
       'Ed Sheeran - Thinking Out Loud',
       '10Honey Honey Mamma Mia Movie Lyrics', 'Elton John - Sacrifice',
       '07Frank Sinatra- Strangers In the Night (with Lyrics)',
       '12-Maroon-5-Sugar-Official', 'Barbra Streisand - Memory',
       'Michael Learns To Rock - The Actor',
       '05Spanish Eyes - Engelbert Humperdinck',
       'Katrina & The Waves - Walking On Sunshine', '08-Danza Kuduro',
       '03 - All Shook U', 'The young ones - Cliff Richard 01-62 - 1(6)GB',
       'Billy Ray Cyrus Achy Breaky Heart Lyrics',
       'Trini Lopez - If I Had A Hammer',
       'Lat Lag Gayee (Race 2) (HQ) (DJMaza.Com)',
       'When the Girl in Your Arms is the Girl in Your Heart',
       '05-Get_Lucky

In [43]:
list(c["Name"].values)

['07- Ottawan - Hands u',
 '12 Just An Illusion The Best Disco... Ever Disc 3',
 "07Unforgettable - Nat 'King' Cole",
 '17Paul Anka - Pa',
 '06Elvis Presley - Wooden Heart (Audio)',
 '11-El Perdón (Forgiveness) - Nicky Jam & Enrique Iglesias _ Video Lyrics',
 'Ed Sheeran - Thinking Out Loud',
 '10Honey Honey Mamma Mia Movie Lyrics',
 'Elton John - Sacrifice',
 '07Frank Sinatra- Strangers In the Night (with Lyrics)',
 '12-Maroon-5-Sugar-Official',
 'Barbra Streisand - Memory',
 'Michael Learns To Rock - The Actor',
 '05Spanish Eyes - Engelbert Humperdinck',
 'Katrina & The Waves - Walking On Sunshine',
 '08-Danza Kuduro',
 '03 - All Shook U',
 'The young ones - Cliff Richard 01-62 - 1(6)GB',
 'Billy Ray Cyrus Achy Breaky Heart Lyrics',
 'Trini Lopez - If I Had A Hammer',
 'Lat Lag Gayee (Race 2) (HQ) (DJMaza.Com)',
 'When the Girl in Your Arms is the Girl in Your Heart',
 '05-Get_Lucky_ft._Pharrell_Williams',
 'American Authors - Believer (Audio)',
 'Grouplove - Tongue Tied [OFFICIAL VI

In [44]:
c.iloc[1:,1:]

,1,2,3,4,5,6,7,8,9,10,...,49143,49144,49145,49146,49147,49148,49149,49150,49151,Name
1,0.658710,0.371280,0.411623,0.673374,0.346420,0.391871,0.725550,0.321411,0.378109,0.716226,...,0.456661,0.707566,0.366236,0.389235,0.699207,0.465402,0.397992,0.708230,0.508073,12 Just An Illusion The Best Disco... Ever Disc 3
2,0.657647,0.377140,0.410791,0.666152,0.357431,0.405265,0.713739,0.333283,0.381318,0.707671,...,0.459422,0.697785,0.370930,0.395673,0.690484,0.460174,0.404935,0.683394,0.509292,07Unforgettable - Nat 'King' Cole
3,0.655098,0.374034,0.411536,0.673598,0.344389,0.396461,0.716538,0.333569,0.382530,0.714318,...,0.460200,0.702658,0.367398,0.387203,0.692189,0.464573,0.403104,0.699180,0.512570,17Paul Anka - Pa
4,0.658427,0.370546,0.414500,0.668260,0.353002,0.405181,0.717680,0.331619,0.379366,0.707514,...,0.458288,0.700104,0.365139,0.393691,0.690549,0.458788,0.404434,0.693158,0.513625,06Elvis Presley - Wooden Heart (Audio)
5,0.667302,0.357038,0.401866,0.680914,0.349486,0.396168,0.731095,0.315610,0.366636,0.716021,...,0.456089,0.704215,0.361105,0.390436,0.707750,0.462431,0.392442,0.713193,0.517829,11-El Perdón (Forgiveness) - Nicky Jam & Enriq...
6,0.654669,0.374403,0.408253,0.674818,0.343410,0.399334,0.711437,0.333646,0.375372,0.709935,...,0.458055,0.703164,0.369992,0.392297,0.696228,0.465701,0.400779,0.701959,0.509043,Ed Sheeran - Thinking Out Loud
7,0.652262,0.367161,0.410409,0.666629,0.351839,0.401410,0.716784,0.333432,0.377879,0.709074,...,0.454474,0.702728,0.370170,0.388050,0.688912,0.461053,0.403136,0.695636,0.511479,10Honey Honey Mamma Mia Movie Lyrics
8,0.650195,0.371556,0.411237,0.672828,0.345390,0.404946,0.715515,0.335522,0.382629,0.711130,...,0.457705,0.702647,0.371199,0.390458,0.688303,0.465748,0.405839,0.695356,0.509190,Elton John - Sacrifice
9,0.654988,0.370943,0.414858,0.669229,0.362382,0.406309,0.713995,0.329151,0.377692,0.708141,...,0.457084,0.700466,0.362203,0.393427,0.687883,0.458741,0.403503,0.687906,0.515427,07Frank Sinatra- Strangers In the Night (with ...
10,0.652666,0.373183,0.412994,0.669954,0.366678,0.411453,0.717636,0.331571,0.377640,0.706825,...,0.459149,0.703757,0.368476,0.397181,0.692901,0.458785,0.399365,0.693161,0.513995,12-Maroon-5-Sugar-Official


In [45]:
import numpy as np

In [46]:
c.index = c.Name

In [47]:
del c['Name']

In [48]:
def recommend(song):
    global c
    from sklearn.metrics import pairwise
    mat = pairwise.pairwise_distances(X=c, Y=c.loc[song], metric='euclidean')
    #np.fill_diagonal(mat, val=2)
    simdf = pd.DataFrame(mat, columns=[song], index = c.index)
    simdf.sort_values(song, inplace=True)
    return simdf

In [50]:
c

,0,1,2,3,4,5,6,7,8,9,...,49142,49143,49144,49145,49146,49147,49148,49149,49150,49151
Name,,,,,,,,,,,,,,,,,,,,,
07- Ottawan - Hands u,0.377123,0.654770,0.375078,0.412867,0.673108,0.342779,0.399932,0.718035,0.333365,0.382433,...,0.333496,0.460302,0.702895,0.370158,0.390452,0.689041,0.463589,0.403052,0.699344,0.510312
12 Just An Illusion The Best Disco... Ever Disc 3,0.372126,0.658710,0.371280,0.411623,0.673374,0.346420,0.391871,0.725550,0.321411,0.378109,...,0.328509,0.456661,0.707566,0.366236,0.389235,0.699207,0.465402,0.397992,0.708230,0.508073
07Unforgettable - Nat 'King' Cole,0.380807,0.657647,0.377140,0.410791,0.666152,0.357431,0.405265,0.713739,0.333283,0.381318,...,0.339167,0.459422,0.697785,0.370930,0.395673,0.690484,0.460174,0.404935,0.683394,0.509292
17Paul Anka - Pa,0.380448,0.655098,0.374034,0.411536,0.673598,0.344389,0.396461,0.716538,0.333569,0.382530,...,0.332566,0.460200,0.702658,0.367398,0.387203,0.692189,0.464573,0.403104,0.699180,0.512570
06Elvis Presley - Wooden Heart (Audio),0.379877,0.658427,0.370546,0.414500,0.668260,0.353002,0.405181,0.717680,0.331619,0.379366,...,0.343811,0.458288,0.700104,0.365139,0.393691,0.690549,0.458788,0.404434,0.693158,0.513625
11-El Perdón (Forgiveness) - Nicky Jam & Enrique Iglesias _ Video Lyrics,0.371358,0.667302,0.357038,0.401866,0.680914,0.349486,0.396168,0.731095,0.315610,0.366636,...,0.324025,0.456089,0.704215,0.361105,0.390436,0.707750,0.462431,0.392442,0.713193,0.517829
Ed Sheeran - Thinking Out Loud,0.384057,0.654669,0.374403,0.408253,0.674818,0.343410,0.399334,0.711437,0.333646,0.375372,...,0.330708,0.458055,0.703164,0.369992,0.392297,0.696228,0.465701,0.400779,0.701959,0.509043
10Honey Honey Mamma Mia Movie Lyrics,0.381323,0.652262,0.367161,0.410409,0.666629,0.351839,0.401410,0.716784,0.333432,0.377879,...,0.337785,0.454474,0.702728,0.370170,0.388050,0.688912,0.461053,0.403136,0.695636,0.511479
Elton John - Sacrifice,0.379593,0.650195,0.371556,0.411237,0.672828,0.345390,0.404946,0.715515,0.335522,0.382629,...,0.340369,0.457705,0.702647,0.371199,0.390458,0.688303,0.465748,0.405839,0.695356,0.509190


Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       49142, 49143, 49144, 49145, 49146, 49147, 49148, 49149, 49150, 49151],
      dtype='object', length=49152)

In [58]:
recommend('12YouTubeOur last summerMamma Mia lyrics') 

/home/ubuntu/anaconda2/envs/py36tf/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


,12YouTubeOur last summerMamma Mia lyrics
Name,
12YouTubeOur last summerMamma Mia lyrics,0.132583
Michael Learns To Rock - The Actor,1.185854
06Elvis Presley - Wooden Heart (Audio),1.197328
07Frank Sinatra- Strangers In the Night (with Lyrics),1.325825
Elton John - Sacrifice,1.328768
Barbra Streisand - Memory,1.350998
10Honey Honey Mamma Mia Movie Lyrics,1.356408
Billy Ray Cyrus Achy Breaky Heart Lyrics,1.368236
12-Maroon-5-Sugar-Official,1.506172


In [59]:
def top3(song):
    return ', '.join(recommend(song).iloc[1:4].index)

In [60]:
top3('12YouTubeOur last summerMamma Mia lyrics')

/home/ubuntu/anaconda2/envs/py36tf/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


'Michael Learns To Rock - The Actor, 06Elvis Presley - Wooden Heart (Audio), 07Frank Sinatra- Strangers In the Night (with Lyrics)'

In [61]:
df = pd.DataFrame(columns=['userpick', 'recommendations'])

In [62]:
c.index

Index(['07- Ottawan - Hands u',
       '12 Just An Illusion The Best Disco... Ever Disc 3',
       '07Unforgettable - Nat 'King' Cole', '17Paul Anka - Pa',
       '06Elvis Presley - Wooden Heart (Audio)',
       '11-El Perdón (Forgiveness) - Nicky Jam & Enrique Iglesias _ Video Lyrics',
       'Ed Sheeran - Thinking Out Loud',
       '10Honey Honey Mamma Mia Movie Lyrics', 'Elton John - Sacrifice',
       '07Frank Sinatra- Strangers In the Night (with Lyrics)',
       '12-Maroon-5-Sugar-Official', 'Barbra Streisand - Memory',
       'Michael Learns To Rock - The Actor',
       '05Spanish Eyes - Engelbert Humperdinck',
       'Katrina & The Waves - Walking On Sunshine', '08-Danza Kuduro',
       '03 - All Shook U', 'The young ones - Cliff Richard 01-62 - 1(6)GB',
       'Billy Ray Cyrus Achy Breaky Heart Lyrics',
       'Trini Lopez - If I Had A Hammer',
       'Lat Lag Gayee (Race 2) (HQ) (DJMaza.Com)',
       'When the Girl in Your Arms is the Girl in Your Heart',
       '05-Get_Lucky

In [63]:
recommendations = []
for song in c.index:
    recommendations.append(top3(song))

/home/ubuntu/anaconda2/envs/py36tf/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/ubuntu/anaconda2/envs/py36tf/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/ubuntu/anaconda2/envs/py36tf/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.resh

In [64]:
df['userpick'] = c.index

In [65]:
df['recommendations'] = recommendations

In [66]:
df.to_csv('results.csv')

In [67]:
df

,userpick,recommendations
0,07- Ottawan - Hands u,"65 Ottawan - Hands u, Ottawan - Hands u, 17Pau..."
1,12 Just An Illusion The Best Disco... Ever Disc 3,"Katrina & The Waves - Walking On Sunshine, 08-..."
2,07Unforgettable - Nat 'King' Cole,07Frank Sinatra- Strangers In the Night (with ...
3,17Paul Anka - Pa,"07- Ottawan - Hands u, 65 Ottawan - Hands u, O..."
4,06Elvis Presley - Wooden Heart (Audio),07Frank Sinatra- Strangers In the Night (with ...
5,11-El Perdón (Forgiveness) - Nicky Jam & Enriq...,10-Pitbull_-_We_Are_One_Ole_Ola_Feat._Jennifer...
6,Ed Sheeran - Thinking Out Loud,"17Paul Anka - Pa, 05-Get_Lucky_ft._Pharrell_Wi..."
7,10Honey Honey Mamma Mia Movie Lyrics,"17Paul Anka - Pa, 07- Ottawan - Hands u, 65 Ot..."
8,Elton John - Sacrifice,"Michael Learns To Rock - The Actor, Barbra Str..."
9,07Frank Sinatra- Strangers In the Night (with ...,"06Elvis Presley - Wooden Heart (Audio), 12-Mar..."


In [69]:
test = a.astype('float32')/255

In [70]:
test = X.reshape((len(X), np.prod(X.shape[1:])))

In [72]:
encoded_imgs = encoder.predict(test)
decoded_imgs = decoder.predict(encoded_imgs)

In [26]:
%pwd

u'/home/k6814/AudioBasedRecommmendation'